In [1]:
import numpy as np
import pandas as pd
import datetime
from tqdm import tqdm 

import warnings 
warnings.filterwarnings('ignore')

In [2]:
def ud(data,n1):
    data["ud"] = np.sign(data["close"]-data['close'].shift(n1))
    
    ud_cumm=list(np.zeros(n1))
    start = None
    cumm = 0
    for i in range(n1,len(data["date"])):
        if start is None:
            ud_cumm.append(data["ud"][i])
            start = data["ud"][i]
            cumm = data["ud"][i]
        else:
            if data["ud"][i] == start:
                cumm += data["ud"][i]
                ud_cumm.append(cumm)
            else:
                ud_cumm.append(data["ud"][i])
                start = data["ud"][i]
                cumm = data["ud"][i]
    data["ud_cumm"] = ud_cumm
    return data

In [3]:
def count(data,n2,n3):
    
    data["start_or_not"]=None
    data["sell_count"] = None
    data["buy_count"] = None
    data["signal"]=None
    data["price_high"] =None
    data["price_low"] =None
    count_buy = 0
    count_sell = 0
    price_low = None
    price_high = None
    buying_begin=False
    selling_begin = False
    last_buy_price=None
    last_sell_price =None
    
    for i in range(0,len(data["date"])):            
        if buying_begin and count_buy<n3: #买入计数
            price_low = min(data["low"][i],price_low) if price_low else data["low"][i]
            if data["close"][i]>=data["high"][i-2] and data["high"][i]>data["high"][i-1]:
                if count_buy == 0:
                    count_buy += 1
                    data.loc[i,("buy_count")] = count_buy
                    last_buy_price = data["close"][i]
                elif count_buy != 0 and data["close"][i]>last_buy_price:
                    count_buy += 1
                    data.loc[i,("buy_count")] = count_buy
                    last_buy_price = data["close"][i]  
            
                    
        if selling_begin and count_sell<n3: #卖出计数
            price_high = max(data["high"][i],price_high) if price_high else data["high"][i]
            if data["close"][i]<=data["low"][i-2] and data["low"][i]<data["low"][i-1]:
                if count_sell == 0:
                    count_sell += 1
                    data.loc[i,("sell_count")] = count_sell
                    last_sell_price = data["close"][i]
                elif count_sell!=0 and data["close"][i] < last_sell_price:
                    count_sell += 1
                    data.loc[i,("sell_count")] = count_sell
                    last_sell_price = data["close"][i]
            
                    
        if data["ud_cumm"][i] == n2: #卖出启动
            data.loc[i,("start_or_not")] =-1
            count_sell = 0 #初始化计数次数
            data.loc[i,("sell_count")] = count_sell
            selling_begin = True
            price_high = None
        elif data["ud_cumm"][i] == -n2: #买入启动
            data.loc[i,("start_or_not")] =1
            count_buy = 0 #初始化计数次数
            data.loc[i,("buy_count")] = count_buy
            buying_begin = True
            price_low = None
                    
        if count_buy == n3:
            data["signal"][i] = 1
            count_buy = 0
            data.loc[i,("price_low")] = price_low
            buying_begin = False
            
        if count_sell == n3:
            data["signal"][i] = -1
            count_sell = 0
            data.loc[i,("price_high")] = price_high
            selling_begin = False

    return data

In [4]:
def signal(data):
    
    day_first_idexs = data.drop_duplicates('date').index.values #取出相同date的第一个日期index 如0 240 480
    day_idexs = np.hstack([day_first_idexs,data.shape[0]]) #加上最后一个index 如0 240 480。。。57840（方便后面取每日区间）
    my_data_dfs = []
    for day_begin, day_end in zip(day_idexs[:-1], day_idexs[1:]):#取每一天的区间
        tmp_df = data.iloc[day_begin: day_end, :]    
        tmp_df["new_signal"]=None
        tmp_df["new_signal"][tmp_df["time"]< '14:40:00'] = tmp_df.signal.shift(1)
        tmp_df["stop or not"] =False
        direction = None
        init = False
        count = 0
        
        for i in range(day_begin, day_end):
            
            if tmp_df["time"][i] == '14:40:00':
                tmp_df.loc[i,("new_signal")] = -count
#                 print("------------------------------------")
            elif tmp_df["time"][i] < '14:40:00':
                if direction ==None and tmp_df["signal"][i] ==1:
                    direction =1
                    price_low = data["price_low"][i]
                    count+=1
#                     print("first")

                elif direction ==None and tmp_df["signal"][i] ==-1 :
                    direction = -1
                    count+=-1
#                     print("second")

                elif direction == 1:     
#                     print(tmp_df["low"][i], price_low)
                    if tmp_df["low"][i] - price_low <= 0 and tmp_df["time"][i] < '14:39:00':
                        tmp_df.loc[i+1,("new_signal")] = -1
                        count-=1
                        tmp_df.loc[i+1,("price_low")] = price_low
                        tmp_df.loc[i+1,("stop or not")] = True
                        direction = -1
#                         print("third")
                    elif tmp_df["signal"][i] ==1:
                        price_low = data["price_low"][i]
                        count+=1
#                         print("fourth")
                    elif tmp_df["signal"][i] ==-1:
                        count-=1
#                         print("fourth2")
                elif direction == -1: 
                    if tmp_df["signal"][i] ==1:
                        price_low = data["price_low"][i]
                        count+=1
                        direction =1
#                         print("fifth")
                    elif tmp_df["signal"][i] ==-1:
                        count-=1
                        direction = -1
#                         print("six")

            
        
        '''
            signal：上述策略中的信号，有1，-1，0 三种取值
            new_signal 针对只能开平多仓的新信号点
            my_signal 在new_signal基础上加上日内平仓 表现为14:40:00 发出-1/1信号
            init是初始化阶段，即第一次进行开仓或者平仓操作，初始化阶段后将init设置为false
            
            
            特殊：由于日内平仓，当时间>14:40:00，就算有signal信号，也不进行开仓操作。
        '''

        my_data_dfs.append(tmp_df)
    """ 拼接得到新DataFrane """
    my_data = pd.concat(my_data_dfs)
    return my_data           

In [5]:
#持仓计算
def position(data, shift_period=1):
    signal = data['new_signal']
    position = signal.fillna(method='ffill').shift(shift_period).fillna(0)
    data['position'] = position
    return data

## 算实际开仓

In [11]:
#读取数据
my_data=pd.read_csv(
    '510300.csv', sep=',', parse_dates=True)
my_data_1 = my_data[my_data["date"]>='2022-01-01']
my_data_1 = my_data_1[my_data_1["dates"]<='2022-06-14 15:00:00']
my_data_1 =my_data_1.reset_index(drop=True)
my_data_1

,date,time,dates,open,high,low,close
0,2022-01-04,09:31:00,2022-01-04 09:31:00,5.025,5.027,5.015,5.016
1,2022-01-04,09:32:00,2022-01-04 09:32:00,5.017,5.017,5.009,5.011
2,2022-01-04,09:33:00,2022-01-04 09:33:00,5.011,5.011,5.006,5.006
3,2022-01-04,09:34:00,2022-01-04 09:34:00,5.004,5.004,4.995,4.997
4,2022-01-04,09:35:00,2022-01-04 09:35:00,4.998,5.001,4.998,5.001
...,...,...,...,...,...,...,...
25195,2022-06-14,14:56:00,2022-06-14 14:56:00,4.235,4.235,4.234,4.235
25196,2022-06-14,14:57:00,2022-06-14 14:57:00,4.234,4.235,4.233,4.235
25197,2022-06-14,14:58:00,2022-06-14 14:58:00,4.234,4.240,4.234,4.239
25198,2022-06-14,14:59:00,2022-06-14 14:59:00,4.239,4.239,4.236,4.239


## 实际开仓（加上6s的频）

In [12]:
my_data_3s=pd.read_csv(
    '510300_12s.csv', sep=',', parse_dates=True)
my_data_3s_1 = my_data_3s[my_data_3s["date"]>='2022-01-01']
my_data_3s_1 = my_data_3s_1[my_data_3s_1["dates"]<='2022-06-14 15:00:00']
my_data_3s_1 = my_data_3s_1[my_data_3s_1["time"]>='09:31:00']
my_data_9s_1 =my_data_3s_1.reset_index(drop=True)
my_data_9s_1

,date,time,dates,open,high,low,close,buy1,bc1,buy2,bc2,sale1,sc1,sale2,sc2
0,2022-01-04,09:31:00,2022-01-04 09:31:00,5.016,5.016,5.015,5.016,5.016,69400,5.015,1740800,5.017,441000,5.018,15000
1,2022-01-04,09:31:12,2022-01-04 09:31:12,5.017,5.017,5.016,5.016,5.015,565700,5.014,55000,5.016,299600,5.017,125700
2,2022-01-04,09:31:24,2022-01-04 09:31:24,5.015,5.016,5.013,5.013,5.013,287500,5.012,15600,5.014,900000,5.015,687900
3,2022-01-04,09:31:36,2022-01-04 09:31:36,5.013,5.013,5.009,5.009,5.009,903800,5.008,365600,5.010,6800,5.011,10800
4,2022-01-04,09:31:48,2022-01-04 09:31:48,5.009,5.011,5.009,5.011,5.011,265100,5.010,907500,5.012,300000,5.013,381700
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
125575,2022-06-14,14:59:12,2022-06-14 14:59:12,4.237,4.237,4.237,4.237,4.236,3200,4.235,1397700,4.237,470700,4.238,377500
125576,2022-06-14,14:59:24,2022-06-14 14:59:24,4.236,4.236,4.235,4.235,4.235,586800,4.234,804300,4.236,21700,4.237,339900
125577,2022-06-14,14:59:36,2022-06-14 14:59:36,4.235,4.235,4.234,4.235,4.235,584800,4.234,801300,4.236,37500,4.237,410300
125578,2022-06-14,14:59:48,2022-06-14 14:59:48,4.236,4.237,4.236,4.237,4.236,1471600,4.235,84100,4.237,1600,4.238,100


In [8]:
#计算新的开平仓收益
def new_earning(data,init_money,init_volume,r0):
    
    '''
        r0:交易单边手续费
        times:每日交易次数  = 固定为3次
        初始化持有金额和仓位（初始持有：两者之和）：
            init_money              初始持有金额
            init_volume             初始持有仓位 
        
        描述变量：
            hold_volume             持有仓位
            all_volume              数量上等于hold_volume 为方便计算引入的变量，目的：比如多仓下平仓，用hold_volume来计算，
                                    判断平仓点为hold_volume=0,而all_volume保留为平仓前的多仓持有仓位
                                    
            yesterday_volume        昨日持仓量
            today_volume            今日持仓量
            change_volume           单次买卖操作变化持仓量

            hold_money              持有金额,初始化为初始持有金额
            all_money               数量上等于hold_money 为方便计算引入的变量
            change_money            单次买卖操作变化金额
            everyday_money          list 每日初始持有金额，第一天为init_money
            
            change_time             单次买卖操作时间（second）
            
            new_price               平仓开仓的价格，all_money/all_volume
            per_time_volume         每次操作的量
            
        标识：     
            buy_signal              买入标识
            sell_signal             卖出标识
            begin_time              每次操作时开仓时间（用来计算每次买卖的所用时长）
            
    '''
    data["yesterday_volume"]=None
    data["today_volume"]=None
    data["change_volume"]=None
    data["all_money"]=None
    data["change_money"]=None
    data["new_position"]=None
    data["new_price"]=None
    data["change_time"]=None
    hold_money=init_money                   #初始持有金额
    all_money=hold_money
    yesterday_volume = init_volume           #昨日及之前持仓
    today_volume = 0                      #今日持仓
    everyday_money=[]                     #每日初始金额
    times =3
    per_time_volume =init_volume/times   #计算每次操作的量
#     print(times,(data[data['new_signal']>0].shape[0]+data[data['new_signal']<0].shape[0]))
    
    day_first_idexs = data.drop_duplicates('date').index.values #取出相同date的第一个日期index
    day_idexs = np.hstack([day_first_idexs,data.shape[0]]) #加上最后一个index （方便后面取每日区间）
    my_data_dfs = []
    for day_begin, day_end in zip(day_idexs[:-1], day_idexs[1:]):#取每一天的区间
        tmp_df = data.iloc[day_begin: day_end, :]   
        #yesterday_volume                  #每日初始持有仓位
        yesterday_volume += today_volume   #每日初始持有仓位 = 昨日的留存的yesterday_volume + 昨日的today_volume
        everyday_money.append(all_money)   #每日初始持有金额计入

        
        #初始化今日量，买卖signal，开仓时间
        today_volume =0                    
        buy_signal = 0
        sell_signal = 0
        begin_time = 0
        
        for i in range(day_begin, day_end):
            if(tmp_df["time"][i]<'14:40:00'): 
                '''
                    在14:40:00前: 
                    买入条件：发出买入信号（new_signal=1），同时今日买入的仓位<初始仓位
                        初始化：发出buy_signal,hold_volume单次初始持仓=0，持有金额=all_money
                                单次所需买入的量为min(per_time_volume，init_volume-today_volume)
                                begin_time = 发出信号的时间
                        
                    卖出条件：发出卖出信号（new_signal=-1），同时昨日持仓>0
                        初始化：发出sell_signal,hold_volume单次卖出的量min(yesterday_volume,per_time_volume)，
                                per_money 单次操作金额=0
                                begin_time = 发出信号的时间
                '''                
                
                if(tmp_df["new_signal"][i]==1 and today_volume<init_volume):           
                    buy_signal=1
                    hold_volume=0
                    hold_money=all_money
                    need_volume=min(per_time_volume,init_volume-today_volume)
                    tmp_df.loc[i,("all_money")]=all_money
                    tmp_df.loc[i,("yesterday_volume")]=yesterday_volume
                    tmp_df.loc[i,("today_volume")]=today_volume
                    begin_time = tmp_df["time"][i]


                elif(tmp_df["new_signal"][i]==-1):        
                    sell_signal=1
                    hold_volume =  min(yesterday_volume,per_time_volume)
                    all_volume = hold_volume
                    per_money = 0
                    tmp_df.loc[i,("all_money")]=all_money
                    tmp_df.loc[i,("yesterday_volume")]=yesterday_volume
                    tmp_df.loc[i,("today_volume")]=today_volume  
                    begin_time = tmp_df["time"][i]
                    
            
            elif(tmp_df["time"][i]=='14:40:00' ):
                '''
                    在14:40:00后: 日内平仓操作
                    买入条件：today_volume+yesterday_volume-init_volume<0
                        初始化：发出buy_signal,hold_volume单次初始持仓=0，持有金额=all_money
                                单次所需买入的量为 init_volume-(today_volume+yesterday_volume)
                                begin_time = 发出信号的时间
                        
                    卖出条件：today_volume+yesterday_volume-init_volume>0
                        初始化：发出sell_signal,hold_volume单次卖出的量，
                                per_money 单次操作金额=0
                                begin_time = 发出信号的时间
                '''
#                 if (buy_signal==1):
#                     hold_money=all_money
#                     buy_signal=0
#                 elif(sell_signal==1):
#                     sell_signal=0
                    
                if(today_volume+yesterday_volume-init_volume>0):  
                    if(yesterday_volume>=today_volume+yesterday_volume-init_volume):
                        hold_volume =  today_volume+yesterday_volume-init_volume
                    else:
                        hold_volume =  yesterday_volume
                    sell_signal=1
                    all_volume = hold_volume
                    per_money = 0
                    tmp_df.loc[i,("all_money")]=all_money
                    tmp_df.loc[i,("yesterday_volume")]=yesterday_volume
                    tmp_df.loc[i,("today_volume")]=today_volume
                    begin_time = tmp_df["time"][i]
                elif(today_volume+yesterday_volume-init_volume<0): 
                    buy_signal=1
                    hold_volume=0
                    hold_money=all_money
                    need_volume = init_volume-(today_volume+yesterday_volume)
                    tmp_df.loc[i,("all_money")]=all_money
                    tmp_df.loc[i,("yesterday_volume")]=yesterday_volume
                    tmp_df.loc[i,("today_volume")]=today_volume
                    begin_time = tmp_df["time"][i]
                     
                
            if(buy_signal ==1):
            
                '''
                    对于每次开仓过程中有三种情况：
                    1.持有金额>卖1价*卖1量+卖2价*卖2量 
                    2.卖1价*卖1量+卖2价*卖2量>=持有金额>卖2价*卖2量 
                    3.持有金额<=卖1价*卖1量  此时以卖1价买入
                    hold_volume 本次买入的仓位 0-->
                    today_volume 今日仓位
                    hold_money 持有的钱(变量)
                    all_money 本次操作之前持有的钱 --> hold_money
                '''
                #计算两档金额
                money_1 = tmp_df["sale1"][i]*tmp_df["sc1"][i]
                money_2 = tmp_df["sale2"][i]*tmp_df["sc2"][i]

                if(need_volume-hold_volume>tmp_df["sc1"][i] and need_volume-hold_volume<=tmp_df["sc1"][i]+tmp_df["sc2"][i]):
                    #需要再扫一档 +二档的一部分
                    need_money = (money_1 + (need_volume-hold_volume-tmp_df["sc1"][i])*tmp_df["sale2"][i])*(1+r0)
                    if(hold_money>=need_money):
                        #钱够扫
                        hold_volume = need_volume
                        hold_money -= need_money
                    else:
                        #钱只够扫一档+二档的小部分， 不能扫满所需二档
                        if(hold_money>money_1*(1+r0)):
                            hold_volume = hold_volume + tmp_df["sc1"][i] + (hold_money - money_1*(1+r0))*(1-r0)/tmp_df["sale2"][i]
                            hold_money = 0
                        else:
                            #钱只够扫一档一部分
                            hold_volume = hold_volume + hold_money*(1-r0)/tmp_df["sale1"][i]
                            hold_money = 0  

                elif(need_volume-hold_volume>tmp_df["sc1"][i]+tmp_df["sc2"][i]):   
                    #需要再扫一档 +二档
                    need_money = (money_1 + money_2)*(1+r0)
                    if(hold_money>need_money):
                        #钱够扫
                        hold_volume += tmp_df["sc1"][i]+tmp_df["sc2"][i]
                        hold_money -= need_money
                    else:
                        #钱只够扫一档+二档的小部分， 不能扫满所需二档
                        if(hold_money>money_1*(1+r0)):
                            hold_volume = hold_volume + tmp_df["sc1"][i] + (hold_money - money_1*(1+r0))*(1-r0)/tmp_df["sale2"][i]
                            hold_money = 0
                        else:
                            #钱只够扫一档一部分
                            hold_volume = hold_volume + hold_money*(1-r0)/tmp_df["sale1"][i]
                            hold_money = 0  

                else:
                    #需要再扫一档 
                    need_money = (need_volume-hold_volume)*tmp_df["sale1"][i]*(1+r0)
                    if(hold_money>need_money):
                        #钱够扫
                        hold_volume = need_volume
                        hold_money -= need_money
                    else:
                        #钱只够扫一档一部分
                        hold_volume = hold_volume + hold_money*(1-r0)/tmp_df["sale1"][i]
                        hold_money = 0  

                #如果上次买入操作在14:39:40前还没结束，那么结束此次买入操作，等到14:40:00进行日内平仓
                if(hold_volume>=need_volume or hold_money== 0 or (tmp_df.loc[i,("time")]>'14:39:40' and tmp_df.loc[i,("time")]<'14:40:00')):
                    today_volume+=hold_volume
                    tmp_df.loc[i,("today_volume")]=today_volume
                    tmp_df.loc[i,("yesterday_volume")]=yesterday_volume
                    tmp_df.loc[i,("change_volume")]=hold_volume
                    tmp_df.loc[i,("change_money")]=all_money-hold_money
                    tmp_df.loc[i,("new_price")]=(all_money-hold_money)/(hold_volume)      
                    all_money = hold_money
                    tmp_df.loc[i,("all_money")]=all_money
                    tmp_df.loc[i,("change_time")]=(datetime.datetime.strptime(tmp_df.loc[i,("time")], "%H:%M:%S")-datetime.datetime.strptime(begin_time, "%H:%M:%S")).seconds
                    buy_signal=0
                    
                
                           
            elif(sell_signal ==1):
                '''
                    对于每次平仓过程中有三种情况：
                    1.持有仓位>买1量+买2量 
                    2.买1量+买2量>=持有仓位>买1量 
                    3.持有仓位<=买1量  此时以买1价都卖出
                    per_money 卖出的钱
                    hold_money/all_money 总持有钱
                    hold_volume 初始为per_time_volume 每次卖出的量 --> 0
                    all_volume 初始的hold_volume --> 0
                '''
                money_1 = tmp_df["buy1"][i]*tmp_df["bc1"][i]
                money_2 = tmp_df["buy2"][i]*tmp_df["bc2"][i]
                
                #如果上次卖出操作在14:39:40前还没结束，那么结束此次卖出操作，等到14:40:00进行日内平仓
                if(hold_volume-tmp_df["bc1"][i]-tmp_df["bc2"][i]>0 and (tmp_df.loc[i,("time")]<='14:39:40' or tmp_df.loc[i,("time")]>='14:40:00')):
                    hold_volume=hold_volume - tmp_df["bc1"][i]-tmp_df["bc2"][i]
                    per_money = per_money+(money_1+money_2)*(1-r0)
                else: 
                    if(tmp_df.loc[i,("time")]<='14:39:40' or tmp_df.loc[i,("time")]>='14:40:00'):
                        if (hold_volume-tmp_df["bc1"][i]-tmp_df["bc2"][i]<=0 and hold_volume-tmp_df["bc1"][i]>0):
                            per_money += (money_1+(hold_volume-tmp_df["bc1"][i])*tmp_df['buy2'][i])*(1-r0)
                        else:
                            per_money +=  hold_volume*tmp_df["buy1"][i]*(1-r0) 
                        # 仓位在卖掉后为0
                        hold_volume=0
                    elif(tmp_df.loc[i,("time")]>'14:39:40'and tmp_df.loc[i,("time")]<'14:40:00'):
                        all_volume = all_volume-hold_volume
                        
                    yesterday_volume -= all_volume
                    tmp_df.loc[i,("yesterday_volume")]=yesterday_volume
                    tmp_df.loc[i,("today_volume")]=today_volume
                    tmp_df.loc[i,("change_volume")]=all_volume
                    tmp_df.loc[i,("change_money")]=per_money
                    hold_money += per_money
                    all_money=hold_money
                    tmp_df.loc[i,("all_money")]=all_money
                    tmp_df.loc[i,("new_price")]=(per_money)/all_volume
                    tmp_df.loc[i,("change_time")]=(datetime.datetime.strptime(data.loc[i,("time")], "%H:%M:%S")-datetime.datetime.strptime(begin_time, "%H:%M:%S")).seconds
                    all_volume=0
                    sell_signal=0
                
            
        my_data_dfs.append(tmp_df)
    """ 拼接得到新DataFrane """
    my_data = pd.concat(my_data_dfs) 
    return my_data,everyday_money

In [9]:
#算每日操作次数 每日操作金额 每日操作平均所需时长
def per_day(data,money,everyday_money):
    '''
       每日操作次数 per_day_count 
       每日操作量 per_day_change_volume
       每日操作金额 per_day_change_money
       每次操作平均所需时长 per_day_change_time
       最大回撤 drawback
    
    '''
#     count_num = data[(data['new_signal']>0) or (data['new_signal']<0)].shape[0] #操作次数
    count_num = data[data['new_signal']>0].shape[0]+data[data['new_signal']<0].shape[0] #操作次数
    total_change_volume = data['change_volume'].dropna().values.sum()
    total_change_money = data['change_money'].dropna().values.sum()
    total_change_time = data['change_time'].dropna().values.sum()
    total_days = data.drop_duplicates('date').index.values.size
    per_day_count = 3
    per_day_change_volume =total_change_volume /total_days
    per_day_change_money = total_change_money /total_days
    per_count_change_time = total_change_time /count_num
    earning = (my_new_situation_earning['all_money'].dropna().values[-1]/money -1)*100
    print(my_new_situation_earning['all_money'].dropna().values[-1],money)
    
    drawback_list = []
    for i in range(0,len(everyday_money)):
        drawback_list.append((everyday_money[i]-min(everyday_money[i:]))/everyday_money[i])
    drawback = max(drawback_list)*100
    return per_day_count,per_day_change_volume,per_day_change_money,per_count_change_time,earning,drawback

# per_day_count,per_day_change_volume,per_day_change_money,per_count_change_time,earning = per_day(my_new_situation_earning,20000000)
# per_day_count,per_day_change_volume,per_day_change_money,per_count_change_time,earning

In [ ]:
performance=[]
drawback=[]
per_money_day=[]
N1 = np.arange(1,11)#11 6 7
N2 = np.arange(1,11)
N3 = np.arange(5,8)
money=[10000000,20000000]
df_par = pd.DataFrame()
df_par["n1"]=np.zeros(600)
df_par["n2"]=np.zeros(600)
df_par["n3"]=np.zeros(600)
df_par["per_day_count"]=np.zeros(600)
df_par["per_day_change_volume"]=np.zeros(600)
df_par["per_day_change_money"]=np.zeros(600)
df_par["per_count_change_time"]=np.zeros(600)
df_par["performance"]=np.zeros(600)
df_par["drawback"]=np.zeros(600)
# print(N1,N2,N3)
m1,m2,m3,m4,m5,m6,m7,m8,m9,m10 =[],[],[],[],[],[],[],[],[],[]
for n1 in N1:
    for n2 in N2:
        for n3 in N3:
            my_data_ud = ud(my_data_1 , n1)
            my_data_count = count(my_data_ud,n2,n3)
            my_data_signal = signal(my_data_count)
            my_new_situation = pd.merge(my_data_9s_1,my_data_signal,on='dates',how='left')
            my_new_situation =my_new_situation.drop(my_new_situation.columns[[15,16,17,18,19,20,21,22,23,24,25,26,27,28,30]], axis=1)
            my_new_situation.rename(columns={'date_x':'date','time_x':'time','open_x':'open','high_x':'high','low_x':'low','close_x':'close'},inplace=True)
                        
            for n4 in money:
                my_new_situation_earning,everyday_money=new_earning(my_new_situation,n4,5000000,0.00005)
                per_day_count,per_day_change_volume,per_day_change_money,per_count_change_time,earning,drawback = per_day(my_new_situation_earning,n4,everyday_money)
                m1.append(int(n1))
                m2.append(int(n2))
                m3.append(int(n3))
                m4.append(int(n4))
                m5.append(per_day_count)
                m6.append(per_day_change_volume)
                m7.append(per_day_change_money)
                m8.append(per_count_change_time)
                m9.append(earning)
                m10.append(drawback)
                print(n1,n2,n3,n4,earning)

df_par["n1"]=m1
df_par["n2"]=m2
df_par["n3"]=m3
df_par["money"]=m4
df_par["per_day_count"]=m5
df_par["per_day_change_volume"]=m6
df_par["per_day_change_money"]=m7
df_par["per_count_change_time"]=m8
df_par["performance"]=m9
df_par["drawback"]=m10
df_par

7999169.374315532 10000000
1 1 5 10000000 -20.008306256844676
17897060.151522323 20000000
1 1 5 20000000 -10.514699242388382
8951542.942924138 10000000
1 1 6 10000000 -10.484570570758622
18881553.48815949 20000000
1 1 6 20000000 -5.592232559202559
9064482.482959716 10000000
1 1 7 10000000 -9.355175170402841
19064482.482959718 20000000
1 1 7 20000000 -4.677587585201415
8408997.367603282 10000000
1 2 5 10000000 -15.910026323967186
17969124.07821429 20000000
1 2 5 20000000 -10.154379608928544
9637547.230428549 10000000
1 2 6 10000000 -3.624527695714519
19431117.101703007 20000000
1 2 6 20000000 -2.84441449148497
9388169.122078514 10000000
1 2 7 10000000 -6.118308779214865
19269892.330558732 20000000
1 2 7 20000000 -3.6505383472063446
9688769.034371572 10000000
1 3 5 10000000 -3.1123096562842845
19505285.58971952 20000000
1 3 5 20000000 -2.473572051402406


In [ ]:
df_par.to_csv("my_result_12s_2022.csv")

## 测试

In [11]:
my_data_ud = ud(my_data_1 , 6)
my_data_count = count(my_data_ud,1,6)
my_data_signal = signal(my_data_count)
my_new_situation = pd.merge(my_data_9s_1,my_data_signal,on='dates',how='left')
my_new_situation =my_new_situation.drop(my_new_situation.columns[[15,16,17,18,19,20,21,22,23,24,25,26,27,28,30]], axis=1)
my_new_situation.rename(columns={'date_x':'date','time_x':'time','open_x':'open','high_x':'high','low_x':'low','close_x':'close'},inplace=True)
n4=10000000
my_new_situation_earning,everyday_money=new_earning(my_new_situation,n4,5000000,0.00005)
per_day_count,per_day_change_volume,per_day_change_money,per_count_change_time,earning,drawback = per_day(my_new_situation_earning,n4,everyday_money)
per_day_count,per_day_change_volume,per_day_change_money,per_count_change_time,earning,drawback

9650797.89363786 10000000


(5,
 6208228.777783154,
 26698670.192696493,
 64.0754716981132,
 -3.4920210636213977,
 10.329532981906283)

In [58]:
n4=10000000
my_new_situation_earning,everyday_money=new_earning(my_new_situation,n4,5000000,0.00005)


0 0 1666666.666666667
sell 0 0 1666666.6666666667 5000000 5000000
0 0 -833333.333333334
buy 0 0 2500000.0 1666666.666666666 5000000
0 0 -1666666.666666667
buy 0 0 833333.3333333334 2499999.9999999995 5000000
0 0 0.0
0 0 375762.0783532681
sell 0 0 2042428.7450199348 3333333.333333333 5000000
0 0 -1666666.666666667
buy 0 0 833333.3333333334 2499999.9999999995 5000000
0 0 833333.333333333
sell 0 0 1666666.6666666667 4166666.6666666665 5000000
0 0 -3333333.333333334
buy 0 0 0 1666666.666666666 5000000
0 0 -1666666.6666666674
buy 0 0 2500000.0 833333.3333333327 5000000
0 0 1666666.666666666
sell 0 0 2500000.0 4166666.6666666665 5000000
0 0 833333.333333333
sell 0 0 1666666.6666666667 4166666.6666666665 5000000
0 0 -2500000.0000000005
buy 0 0 1666666.6666666667 833333.3333333327 5000000
0 0 833333.333333333
sell 0 0 1666666.6666666667 4166666.6666666665 5000000
0 0 -833333.3333333335
buy 0 0 833333.3333333334 3333333.333333333 5000000
0 0 833333.333333333
sell 0 0 2500000.0 3333333.333333333

In [71]:
my_new_situation_earning[(my_new_situation_earning["yesterday_volume"]>0)|(my_new_situation_earning["today_volume"]>0)|(my_new_situation_earning["all_money"]>0)][100:150]

,date,time,dates,open,high,low,close,buy1,bc1,buy2,...,sc2,new_signal,yesterday_volume,today_volume,change_volume,all_money,change_money,new_position,new_price,change_time
24150,2022-01-24,11:13:00,2022-01-24 11:13:00,4.791,4.791,4.791,4.791,4.790,72400,4.789,...,1867200,1,4e+06,2e+06,1e+06,5.21656e+06,4.79221e+06,None,4.79221,0
24619,2022-01-24,13:50:00,2022-01-24 13:50:00,4.779,4.779,4.779,4.779,4.778,346300,4.777,...,2157300,-1,3e+06,2e+06,1e+06,9.99366e+06,4.77711e+06,None,4.77711,0
24633,2022-01-24,13:52:00,2022-01-24 13:52:00,4.778,4.778,4.778,4.778,4.777,804700,4.776,...,543000,-1,2e+06,2e+06,1e+06,1.47702e+07,4.77657e+06,None,4.77657,0
24948,2022-01-24,14:37:00,2022-01-24 14:37:00,4.781,4.781,4.781,4.781,4.781,610100,4.780,...,1340500,1,2e+06,3e+06,1e+06,9.98701e+06,4.78322e+06,None,4.78322,0
25971,2022-01-25,13:04:00,2022-01-25 13:04:00,4.730,4.730,4.730,4.730,4.730,18200,4.729,...,455300,-1,5e+06,0,None,9.98701e+06,None,None,None,None
25972,2022-01-25,13:04:09,2022-01-25 13:04:09,4.731,4.731,4.731,4.731,4.730,327500,4.729,...,297300,NaN,4e+06,0,1e+06,1.47161e+07,4.72907e+06,None,4.72907,9
26314,2022-01-25,13:53:00,2022-01-25 13:53:00,4.709,4.709,4.709,4.709,4.707,168300,4.706,...,1673100,-1,4e+06,0,None,1.47161e+07,None,None,None,None
26316,2022-01-25,13:53:18,2022-01-25 13:53:18,4.709,4.709,4.709,4.709,4.708,24000,4.707,...,1517200,NaN,3e+06,0,1e+06,1.94222e+07,4.70616e+06,None,4.70616,18
26524,2022-01-25,14:23:00,2022-01-25 14:23:00,4.709,4.709,4.709,4.709,4.708,27900,4.707,...,1870400,-1,3e+06,0,None,1.94222e+07,None,None,None,None
26526,2022-01-25,14:23:18,2022-01-25 14:23:18,4.709,4.709,4.709,4.709,4.708,70100,4.707,...,342400,NaN,2e+06,0,1e+06,2.41291e+07,4.70686e+06,None,4.70686,18


In [31]:
my_new_situation_earning[31668:31693]

,date,time,dates,open,high,low,close,buy1,bc1,buy2,...,sc2,new_signal,yesterday_volume,today_volume,change_volume,all_money,change_money,new_position,new_price,change_time
31668,2022-01-28,14:40:27,2022-01-28 14:40:27,4.576,4.576,4.576,4.576,4.576,86300,4.573,...,772700,NaN,1e+06,4e+06,3e+06,1.0167e+07,1.37331e+07,None,4.57769,27
31669,2022-01-28,14:40:36,2022-01-28 14:40:36,4.573,4.573,4.573,4.573,4.572,18000,4.571,...,301600,NaN,None,None,None,None,None,None,None,None
31670,2022-01-28,14:40:45,2022-01-28 14:40:45,4.572,4.572,4.572,4.572,4.571,1200,4.570,...,287700,NaN,None,None,None,None,None,None,None,None
31671,2022-01-28,14:40:54,2022-01-28 14:40:54,4.572,4.572,4.572,4.572,4.570,94500,4.569,...,232300,NaN,None,None,None,None,None,None,None,None
31672,2022-01-28,14:41:00,2022-01-28 14:41:00,4.571,4.571,4.571,4.571,4.568,1069600,4.567,...,893000,None,0,4e+06,1e+06,2.75764e+07,1.74094e+07,None,17.4094,60
31673,2022-01-28,14:41:09,2022-01-28 14:41:09,4.570,4.570,4.570,4.570,4.569,7900,4.568,...,7900,NaN,None,None,None,None,None,None,None,None
31674,2022-01-28,14:41:18,2022-01-28 14:41:18,4.570,4.570,4.570,4.570,4.569,13500,4.568,...,202000,NaN,None,None,None,None,None,None,None,None
31675,2022-01-28,14:41:27,2022-01-28 14:41:27,4.569,4.569,4.569,4.569,4.568,527900,4.567,...,280600,NaN,None,None,None,None,None,None,None,None
31676,2022-01-28,14:41:36,2022-01-28 14:41:36,4.570,4.570,4.570,4.570,4.569,300,4.568,...,540600,NaN,None,None,None,None,None,None,None,None
31677,2022-01-28,14:41:45,2022-01-28 14:41:45,4.568,4.568,4.568,4.568,4.567,300700,4.566,...,5400,NaN,None,None,None,None,None,None,None,None
